ULTIMATELY DROPPING MAF VAE. THOUGHT TO INCLUDE ONLY FIXED MUTATIONS SO THEY COULD MAP TO SINGLE CELL DATA FROM TAHOE100M, BUT THOSE ARENT SHARED ACROSS CELL LINES, WAY TOO SPARSE FOR VAE.

In [125]:
import scanpy as sc
import pandas as pd
import anndata as ad
import pyarrow.dataset as ds
import gcsfs
import requests
from io import StringIO
import re

In [126]:
fs = gcsfs.GCSFileSystem()

In [127]:
gcp_base_path = "gs://arc-ctc-tahoe100/2025-02-25/"

In [128]:
files = fs.ls(gcp_base_path)
print("Available files:")
for file in files:
    print(f"  {file}")

Available files:
  arc-ctc-tahoe100/2025-02-25/
  arc-ctc-tahoe100/2025-02-25/README.md
  arc-ctc-tahoe100/2025-02-25/h5ad
  arc-ctc-tahoe100/2025-02-25/metadata
  arc-ctc-tahoe100/2025-02-25/tutorial


In [129]:
infile = "/".join([gcp_base_path.rstrip("/"), 'metadata', 'obs_metadata.parquet'])
infile

'gs://arc-ctc-tahoe100/2025-02-25/metadata/obs_metadata.parquet'

In [130]:
# read a subset of the metadata
# Chain it all together
unique_cells = set(ds.dataset(infile, filesystem=fs, format="parquet").to_table(columns=['cell_name']).to_pandas()['cell_name'].unique())
cleaned_cells = [re.sub(r'[\s\-]', '', name).upper() for name in unique_cells]



In [131]:
cleaned_cells

['HS578T',
 'HOP62',
 'SW480',
 'NCIH2030',
 'LOXIMVI',
 'NCIH1792',
 'HS766T',
 'RPMI7951',
 'PANC1',
 'SHP77',
 'CFPAC1',
 'CHP212',
 'SW48',
 'ASPC1',
 'HCT15',
 'NCIH661',
 'NCIH460',
 'A498',
 'KATOIII',
 'HEPG2/C3A',
 'SNU423',
 'A172',
 'BT474',
 'SKMEL2',
 'LS180',
 'PANC03.27',
 'AN3CA',
 'SW1271',
 'NCIH596',
 'SW1417',
 'C33A',
 'NCIH1573',
 'A549',
 'MIAPACA2',
 'RKO',
 'COLO205',
 'C32',
 'SNU1',
 'HEC1A',
 'NCIH2347',
 'J82',
 'HTERTHPNE',
 'NCIH2122',
 'A427',
 'LOVO',
 'NCIH23',
 'HT29',
 'H4',
 'SW900',
 'SW1088']

In [132]:
unique_cells

{'A-172',
 'A-427',
 'A498',
 'A549',
 'AN3 CA',
 'AsPC-1',
 'BT-474',
 'C-33 A',
 'C32',
 'CFPAC-1',
 'CHP-212',
 'COLO 205',
 'H4',
 'HCT15',
 'HEC-1-A',
 'HOP62',
 'HS-578T',
 'HT-29',
 'HepG2/C3A',
 'Hs 766T',
 'J82',
 'KATO III',
 'LOX-IMVI',
 'LS 180',
 'LoVo',
 'MIA PaCa-2',
 'NCI-H1573',
 'NCI-H1792',
 'NCI-H2030',
 'NCI-H2122',
 'NCI-H23',
 'NCI-H2347',
 'NCI-H460',
 'NCI-H596',
 'NCI-H661',
 'PANC-1',
 'Panc 03.27',
 'RKO',
 'RPMI-7951',
 'SHP-77',
 'SK-MEL-2',
 'SNU-1',
 'SNU-423',
 'SW 1088',
 'SW 1271',
 'SW 900',
 'SW1417',
 'SW48',
 'SW480',
 'hTERT-HPNE'}

In [133]:
base_url = "https://depmap.org/portal/api/download/files"

# Try to get available endpoints or documentation
response = requests.get(base_url)
files = pd.read_csv(StringIO(response.text))

In [134]:
mutations_row = files[files['filename'].str.contains('OmicsSomaticMutations.csv', na=False) 
                      & files['release'].str.contains('25Q2', na=False)]

model_row = (files[files['filename'].str.contains('Model.csv')
            & files['release'].str.contains('25Q2', na=False)])

if mutations_row.empty:
    print("MAF file not found")
else:
    maf_url = mutations_row['url'].iloc[0]  

if model_row.empty:
    print("Model file not found")
else:
    model_url = model_row['url'].iloc[0]

In [135]:
response_maf = requests.get(maf_url, stream=True)
response_model = requests.get(model_url)

In [136]:
model = pd.read_csv(StringIO(response_model.text))

In [137]:
id_dict = dict(zip(model['StrippedCellLineName'], model['ModelID']))
id_reverse = {model_id: cell_name for cell_name, model_id in id_dict.items()}

In [138]:
lines = response_maf.iter_lines(decode_unicode=True)

header = next(lines)
header = header.decode('utf-8')
header_cols = header.split(',')
header_cols.append('CellName')

id_pos = header_cols.index('ModelID')
print(header_cols)
print(id_pos)


['Chrom', 'Pos', 'Ref', 'Alt', 'AF', 'DP', 'RefCount', 'AltCount', 'GT', 'PS', 'VariantType', 'VariantInfo', 'DNAChange', 'ProteinChange', 'HugoSymbol', 'Exon', 'Intron', 'EnsemblGeneID', 'EnsemblFeatureID', 'HgncName', 'HgncFamily', 'UniprotID', 'DbsnpRsID', 'GcContent', 'LofGeneName', 'LofGeneId', 'LofNumberOfTranscriptsInGene', 'LofPercentOfTranscriptsAffected', 'NMD', 'MolecularConsequence', 'VepImpact', 'VepBiotype', 'VepHgncID', 'VepExistingVariation', 'VepManeSelect', 'VepENSP', 'VepSwissprot', 'Sift', 'Polyphen', 'GnomadeAF', 'GnomadgAF', 'VepClinSig', 'VepSomatic', 'VepPliGeneValue', 'VepLofTool', 'OncogeneHighImpact', 'TumorSuppressorHighImpact', 'TranscriptLikelyLof', 'Brca1FuncScore', 'CivicID', 'CivicDescription', 'CivicScore', 'LikelyLoF', 'HessDriver', 'HessSignature', 'RevelScore', 'PharmgkbId', 'DidaID', 'DidaName', 'GwasDisease', 'GwasPmID', 'GtexGene', 'ProveanPrediction', 'AMClass', 'AMPathogenicity', 'Rescue', 'RescueReason', 'ModelID', 'Hotspot', 'EntrezGeneID', '

In [139]:
matching_rows = []
target_ids = {id_dict[cell] for cell in cleaned_cells if cell in id_dict}
for line_bytes in lines:
    line = line_bytes.decode('utf-8')
    
    columns = line.split(',')
    
    if columns[67] in target_ids:
        model_id = columns[67]
        cell_name = id_reverse[model_id] 
        columns.append(cell_name)
        matching_rows.append(columns)


In [140]:
print(len(matching_rows))
print(matching_rows[:500])

32786
[['chr1', '939091', 'G', 'A', '0.5', '30', '15', '15', '0/1', '', 'SNV', 'missense_variant', 'ENST00000616016.5:c.1019G>A', 'p.C340Y', 'SAMD11', '6/14', '', 'ENSG00000187634', 'ENST00000616016', 'sterile alpha motif domain containing 11', 'Sterile alpha motif domain containing', '', '', '0.6608478802992519', '', '', '', '', '', '', 'MODERATE', 'protein_coding', 'HGNC:28706', '', 'NM_001385641.1', 'ENSP00000478421', '', 'deleterious_low_confidence(0.02)', 'possibly_damaging(0.75)', '0.0', '0.0', '', '', '0.0', '', 'False', 'False', '', '', '', '', '', 'False', 'False', '', '0.306', '', '', '', '', '', '', 'Damaging', '', '', 'False', '', 'ACH-000669', 'False', '148398', 'SW900'], ['chr1', '942852', 'C', 'A', '0.419', '51', '31', '20', '0/1', '', 'SNV', 'missense_variant', 'ENST00000616016.5:c.1847C>A', 'p.A616D', 'SAMD11', '11/14', '', 'ENSG00000187634', 'ENST00000616016', 'sterile alpha motif domain containing 11', 'Sterile alpha motif domain containing', '', '', '0.7431421446384

In [141]:
MAF_targetIDs = pd.DataFrame( data=matching_rows, columns=header_cols)

In [142]:
print(f"DataFrame shape: {MAF_targetIDs.shape}")
print(f"First few rows:")
print(MAF_targetIDs.head())
print(f"Last column (should be CellName): {MAF_targetIDs.columns[-1]}")
print(f"Unique cell lines: {MAF_targetIDs['CellName'].nunique()}")

DataFrame shape: (32786, 71)
First few rows:
  Chrom     Pos Ref Alt     AF  DP RefCount AltCount   GT PS  ... GtexGene  \
0  chr1  939091   G   A    0.5  30       15       15  0/1     ...            
1  chr1  942852   C   A  0.419  51       31       20  0/1     ...            
2  chr1  952450   C   T  0.467  44       24       20  0/1     ...            
3  chr1  960766   G   A   0.96  22        0       22  1|1     ...            
4  chr1  961336  GC   G  0.409  22       13        9  0/1     ...            

  ProveanPrediction            AMClass AMPathogenicity Rescue RescueReason  \
0          Damaging                                     False                
1           Neutral                                     False                
2          Damaging  likely_pathogenic          0.7253  False                
3           Neutral      likely_benign          0.2713  False                
4                                                       False                

      ModelID Hot

In [143]:
MAF_targetIDs.columns

Index(['Chrom', 'Pos', 'Ref', 'Alt', 'AF', 'DP', 'RefCount', 'AltCount', 'GT',
       'PS', 'VariantType', 'VariantInfo', 'DNAChange', 'ProteinChange',
       'HugoSymbol', 'Exon', 'Intron', 'EnsemblGeneID', 'EnsemblFeatureID',
       'HgncName', 'HgncFamily', 'UniprotID', 'DbsnpRsID', 'GcContent',
       'LofGeneName', 'LofGeneId', 'LofNumberOfTranscriptsInGene',
       'LofPercentOfTranscriptsAffected', 'NMD', 'MolecularConsequence',
       'VepImpact', 'VepBiotype', 'VepHgncID', 'VepExistingVariation',
       'VepManeSelect', 'VepENSP', 'VepSwissprot', 'Sift', 'Polyphen',
       'GnomadeAF', 'GnomadgAF', 'VepClinSig', 'VepSomatic', 'VepPliGeneValue',
       'VepLofTool', 'OncogeneHighImpact', 'TumorSuppressorHighImpact',
       'TranscriptLikelyLof', 'Brca1FuncScore', 'CivicID', 'CivicDescription',
       'CivicScore', 'LikelyLoF', 'HessDriver', 'HessSignature', 'RevelScore',
       'PharmgkbId', 'DidaID', 'DidaName', 'GwasDisease', 'GwasPmID',
       'GtexGene', 'ProveanPrediction'

In [154]:
gene_cellline_counts = MAF_targetIDs.groupby('HugoSymbol')['CellName'].nunique()

In [155]:
highly_shared = gene_cellline_counts[gene_cellline_counts > 5]

In [157]:
highly_shared.index

Index(['ABCA12', 'ABCA13', 'ABCA2', 'ABCB5', 'ABCC12', 'ABCC3', 'ABI3BP',
       'ACACA', 'ACACB', 'ACAN',
       ...
       'ZNF727', 'ZNF729', 'ZNF8', 'ZNF804B', 'ZNF831', 'ZP4', 'ZRANB3',
       'ZSWIM5', 'ZSWIM8', 'ZZEF1'],
      dtype='object', name='HugoSymbol', length=783)

In [160]:
set_for_VAE = MAF_targetIDs[MAF_targetIDs['HugoSymbol'].isin(list(highly_shared.index))]

In [161]:
set_for_VAE

,Chrom,Pos,Ref,Alt,AF,DP,RefCount,AltCount,GT,PS,...,GtexGene,ProveanPrediction,AMClass,AMPathogenicity,Rescue,RescueReason,ModelID,Hotspot,EntrezGeneID,CellName
9,chr1,1042055,G,A,0.544,45,20,25,0/1,,...,,Neutral,,,False,,ACH-000932,False,375790,SNU1
10,chr1,1043598,GC,G,0.151,51,44,7,0/1,,...,,,,,False,,ACH-000890,False,375790,SW1271
11,chr1,1048197,A,G,0.571,48,21,27,0/1,,...,,Neutral,,,False,,ACH-000943,False,375790,RKO
12,chr1,1051492,G,C,0.979,46,0,46,1|1,,...,,Neutral,,,False,,ACH-000900,False,375790,NCIH23
59,chr1,1922348,C,CA,0.446,57,32,25,0/1,,...,,,,,False,,ACH-000954,False,85452,HEC1A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32779,chrX,154460407,C,G,0.458,22,12,10,0/1,,...,,Neutral,likely_benign,0.1027,False,,ACH-000757,False,55558,A427
32780,chrX,154467241,G,T,0.488,83,42,41,0/1,,...,,Neutral,ambiguous,0.4584,False,,ACH-000875,False,55558,NCIH2347
32781,chrX,154904925,G,GT,0.445,82,46,36,0/1,,...,,,,,False,,ACH-001039,False,2157,COLO205
32782,chrX,154931007,A,T,0.987,76,0,76,1|1,,...,,Neutral,likely_benign,0.1023,False,,ACH-000916,False,2157,NCIH1573


In [163]:
highAF = set_for_VAE[set_for_VAE['AF'].astype(float) > .9]

In [164]:
highAF

,Chrom,Pos,Ref,Alt,AF,DP,RefCount,AltCount,GT,PS,...,GtexGene,ProveanPrediction,AMClass,AMPathogenicity,Rescue,RescueReason,ModelID,Hotspot,EntrezGeneID,CellName
12,chr1,1051492,G,C,0.979,46,0,46,1|1,,...,,Neutral,,,False,,ACH-000900,False,375790,NCIH23
137,chr1,6197724,CT,C,0.973,36,0,36,1|1,,...,,,,,False,,ACH-001333,False,6146,C33A
138,chr1,6197724,CT,C,0.975,35,0,35,1|1,,...,,,,,False,,ACH-000940,False,6146,AN3CA
168,chr1,7664939,G,T,0.98,48,0,48,1|1,,...,,Damaging,ambiguous,0.3605,False,,ACH-000890,False,23261,SW1271
242,chr1,11519374,CG,TT,0.983,57,0,57,1|1,,...,,,,,False,,ACH-000890,False,57540,SW1271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32735,chrX,124641842,C,A,0.992,126,0,126,1|1,,...,,Neutral,likely_benign,0.0562,False,,ACH-000916,False,10178,NCIH1573
32760,chrX,145824731,C,G,0.992,125,0,125,1|1,,...,,Neutral,likely_benign,0.0645,False,,ACH-000916,False,84631,NCIH1573
32761,chrX,148662256,C,T,0.988,159,1,158,1|1,,...,,,,,False,,ACH-000916,False,2334,NCIH1573
32763,chrX,148991291,C,CA,0.989,89,0,89,1|1,,...,,,,,False,,ACH-000916,False,2334,NCIH1573


In [165]:
highAF.groupby('HugoSymbol')['CellName'].nunique()

HugoSymbol
ABCA12    1
ABCC12    1
ACACB     1
ACAN      1
ACSL6     2
         ..
ZNF407    2
ZNF469    2
ZNF536    1
ZNF729    1
ZSWIM8    1
Name: CellName, Length: 322, dtype: int64